In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from google.colab import drive
import sys
from pathlib import Path
drive.mount("/content/drive", force_remount=True)
colabase = Path('/content/drive/My Drive/Colab Notebooks/')
course = Path('/content/drive/My Drive/deeplearning-ff/')
dlff = Path('/content/drive/My Drive/deeplearning-ff/')
sys.path.append(str(dlff))
# cd {dlff}

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Does nn.Conv2d init work well?

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=21)

In [0]:
#export
from exports.lg_02 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [0]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [0]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [7]:
n,*_ = x_train.shape
c = y_train.max()+1
nh = 32
n,c

(50000, tensor(10))

In [0]:
l1 = nn.Conv2d(1, nh, 5)

In [0]:
x = x_valid[:100]

In [10]:
x.shape

torch.Size([100, 1, 28, 28])

In [0]:
def stats(x): return x.mean(),x.std()

In [12]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [15]:
stats(l1.weight),stats(l1.bias)

((tensor(-0.0054, grad_fn=<MeanBackward0>),
  tensor(0.1140, grad_fn=<StdBackward0>)),
 (tensor(0.0220, grad_fn=<MeanBackward0>),
  tensor(0.1176, grad_fn=<StdBackward0>)))

In [0]:
t = l1(x)

In [17]:
stats(t) # std is not really 1 

(tensor(0.0083, grad_fn=<MeanBackward0>),
 tensor(0.6030, grad_fn=<StdBackward0>))

In [20]:
init.kaiming_normal_(l1.weight, a=1.) # leaky relu cooef , a = 1 leak 1
stats(l1(x))

(tensor(0.0350, grad_fn=<MeanBackward0>),
 tensor(1.0041, grad_fn=<StdBackward0>))

In [0]:
import torch.nn.functional as F

In [0]:
# regualar ReLU as a=0
def f1(x,a=0): return F.leaky_relu(l1(x),a)

In [24]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5761, grad_fn=<MeanBackward0>),
 tensor(1.1041, grad_fn=<StdBackward0>))

In [26]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2067, grad_fn=<MeanBackward0>),
 tensor(0.3984, grad_fn=<StdBackward0>))

In [27]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [28]:
# receptive field size
rec_fs = l1.weight[0,0].numel()
rec_fs

25

In [29]:
nf,ni,*_ = l1.weight.shape
nf,ni

(32, 1)

In [30]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in,fan_out

(25, 800)

In [0]:
def gain(a): return math.sqrt(2.0 / (1 + a**2))

In [32]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [35]:
torch.zeros(10000).uniform_(-1,1).std() # simulate uniform -1,1 and check std

tensor(0.5752)

In [36]:
1/math.sqrt(3.)

0.5773502691896258

In [0]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

In [38]:
kaiming2(l1.weight, a=0);
stats(f1(x))

(tensor(0.5287, grad_fn=<MeanBackward0>),
 tensor(0.9647, grad_fn=<StdBackward0>))

In [39]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.2196, grad_fn=<MeanBackward0>),
 tensor(0.4114, grad_fn=<StdBackward0>))

In [0]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [0]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [0]:
y = y_valid[:100].float()

In [43]:
t = m(x)
stats(t)

(tensor(-0.0156, grad_fn=<MeanBackward0>),
 tensor(0.0077, grad_fn=<StdBackward0>))

In [0]:
# Hmm, var went to close to zero 

In [0]:
l = mse(t,y)
l.backward()

In [45]:
stats(m[0].weight.grad)

(tensor(-0.0037), tensor(0.0293))

In [0]:
init.kaiming_uniform_??

In [0]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [48]:
t = m(x)
stats(t)

(tensor(-0.1466, grad_fn=<MeanBackward0>),
 tensor(0.2020, grad_fn=<StdBackward0>))

In [0]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(0.0093), tensor(0.4231))

## Export

In [0]:
!./notebook2script.py 02a_why_sqrt5.ipynb